<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/01_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストのBoW (bag-of-words)モデル
* テキストを単語のmultisetとして表現したもの
  * もはや単語の出現回数でしかテキストを区別できない。
  * つまり、語順は無視される。
  * それでも、それなりに興味深い分析が実現できる。


## 前置き

### 参考書

* 授業全体の参考書: 岡﨑、荒瀬、鈴木、鶴岡、宮尾著 『IT Text 自然言語処理の基礎』（オーム社）
  * https://www.ohmsha.co.jp/book/9784274229008/
* 本日の参考書: C. D. Manning, P. Raghavan & H. Schütze. Introduction to Information Retrieval.
  * https://nlp.stanford.edu/IR-book/html/htmledition/irbook.html

### prerequisites

* この授業では、Pythonのコーディングの基礎は習得済みであることを前提します。
* また、NumPyやscikit-learnの基本的な使い方は習得済みであることを前提します。
* PyTorch（Hugging Faceのライブラリを使う時に必要）は、授業で説明します。

### NLPの歴史
* bag-of-words(BoW)は、テキストをモデル化する方法の、一つ。
* 近年のNLPの歴史＝テキストのモデル化がBoWから文脈化埋め込みに変化した歴史
  * BoW → word2vec → contexualized word embedding
* word2vec
  * 前後にどんな単語が出現するかによらず・・・
  * 一単語に一つのベクトルを固定的に割り振る。
  * 今は言語モデルの入力ベクトルとして使うだけ(embedding layer)。
* 文脈化埋め込み (contextualized word embedding)
  * LLMs (large language models) は、文脈化埋め込みを実現する手法の一つ。
  * 他にも文脈化埋め込みを実現する方法はある(cf. [sequence memoizer](https://dl.acm.org/doi/10.1145/1897816.1897842))。
    * https://x.com/yeewhye/status/1753267400676463054
* NLPの歴史の参考資料
  * スタンフォード大のNLPの授業がいかに大きく内容を変えているか、調べてみよう。
  * http://web.stanford.edu/class/cs224n/index.html の"Previous offerings"

### 用語集

* **単語(word)**
  * テキストを構成する最小の単位。
  * LLMの世界では、単語をさらに分割したサブワード(subword)が最小の単位になる。

* **語彙(vocabulary)**
  * あるコーパスに出現する単語またはサブワードの集合のこと。


* **トークン(token)**
  * 単語(word)、またはサブワード(subword)の、一回一回の出現のこと。
  * 例えば、このセルで「この」という単語は5回現れている。
  * このことを、このセルでは「この」という単語のトークンが5個ある、と言い表す。


* **テキスト(text)**
  * トークン列のこと。
  * 当然、同じ単語のトークンが複数回現れることもある。
  * テキストのことを、文書(document)と呼ぶこともある。
* **コーパス(corpus)**
  * テキストの集合のこと。

## テキストのBoW(bag-of-words)モデル
* **bag-of-wordsモデル**とは、テキストを定量的に表現する手法のひとつ。
  * 他にもテキストを定量的に表現する手法はある。
* bag-of-wordsモデルにおいては、トークンの**出現順序が無視される**。
* つまり、テキストを、バッグに入ったトークンの集まりのようにモデリングする（下図参照）。
  * 言い換えれば、テキストを単語の**multiset**として扱うのがbag-of-wordsモデルである。


![bag-of-words.png](https://raw.githubusercontent.com/tomonari-masada/course2024-nlp/main/bag-of-words.png)

* 上の図は下記のWebページより。
 * https://dudeperf3ct.github.io/lstm/gru/nlp/2019/01/28/Force-of-LSTM-and-GRU/

### テキストのベクトル表現の歴史
* 最近では、単語またはsubwordのベクトル表現を利用して、テキストのベクトル表現を作る。
  * word2vec以降の流れ。
* 単語またはsubwordのベクトル表現は、埋め込み(embedding)と呼ばれる。
  * 分散表現(distributive representation)とも呼ばれるが、最近あまり使わない呼び方。
* word embeddingやsubword embeddingを元にして、テキストをembedするのが、今は主流。

### BoWはまだ現役か？
* 論文では今でも、baselineとして、TF-IDFやBM25など、BoWが引き合いに出されることがある。
  * 新しい手法を考え出してもBoWに勝てなければ意味がない。
* そのため、最初にBoWについて簡単に説明しておく。
  * BM25 https://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
* ただし、最近でもRAGではBM25をdense retrievalと組み合わせることもあるので、まだ現役と言える。
  * RAG = retrieval augmented generation

## BoWモデル1: Word count
* テキストは、各単語の出現回数を要素とするベクトルとして表現できる。

### scikit-learnのCountVectorizer
* 各テキストは、半角スペースでつながれた単語の列として準備しておく。
* CountVectorizerのインスタンスを作り、テキスト集合にfitさせる。
  * 語彙の抽出と、出現回数の集計が実行される。
* そして他の任意のテキスト集合をtransformする。
  * この使い方は、scikit-learnにおける他の前処理のときと同様。
  * fitメソッドに与えたテキスト集合の語彙が使われる。


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

* コーパス（＝テキストの集合）を用意する。

In [ ]:
corpus = [
    "This document is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Where is the fourth one?"
]

* CountVectorizerをデフォルトの設定で使う。

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

* テキストのBoW表現の確認
 * 疎なベクトルとして得られることに注意。

In [ ]:
print(X)

In [ ]:
type(X)

* 疎な表現を通常の密な表現（NumPyのndarray）にする。

In [ ]:
X_dense = X.toarray()
X_dense

### 語彙を確認
* 先頭の大文字は自動的に小文字に変換されていることが分かる。
* ピリオドや疑問符は削除されている。

In [ ]:
vectorizer.vocabulary_

In [ ]:
type(vectorizer.vocabulary_)

* 語彙を取得

In [ ]:
vocab = vectorizer.get_feature_names_out()
vocab

### 新しいテキストをベクトルに変換
* sklearnでよくやるように、transformメソッドを使う。

In [ ]:
new_doc = ["This is the new document."]
new_vectors = vectorizer.transform(new_doc)
new_vectors.toarray()

* 新出の単語は無視される点に注意
  * 問： 上の例題で、どれが新出単語か？
* OoV (out-of-vocabulary) の問題
  * この問題は、NLPの世界では、超重要な問題。
  * 今は、サブワード(subword)の利用により、OoV問題を回避する。

## BoWモデル2: TF-IDF
* テキストをベクトル化する古典的な手法。
* TF-IDFは、TFとIDFの積である。

### TF (term frequency)
* TFとは、各々の単語がテキストのなかで出現する回数。word countと同義。
  * 出現回数を、そのテキストの長さで割ったものをTFと呼ぶこともある。
  * テキストの長さとは、テキストに含まれるトークンの総数。
* テキストのなかで頻出する単語ほどTFは大きくなる。

### IDF (inverse document frequency)
* IDFとは、DFの逆数。
* DFとは、ある単語が含まれるテキストの数。
  * ある単語が含まれるテキスト数を全テキスト数で割ったものをDFと呼ぶこともある。
* テキスト集合のなかで稀少な単語ほどIDFは大きくなる。

### TF-IDF (term frequency–inverse document frequency)
* TF-IDFは、TFとIDFの積。
* 積を求める前に、TFのルートもしくは対数をとったり、IDFのルートもしくは対数をとったりする。
  * 大きめの値が、効きすぎないようにする。
  * 対数をとるときは、ゼロの対数をとることにならないように、1を足したりする。

### TF-IDFの式の例

\begin{align}
x_{d,w} = \frac{n_{d,w}}{n_d} \cdot ( 1 + \log\frac{m}{m_w}) \tag{1}
\end{align}

where

 * $n_{d,w}$ is the frequency of the word $w$ in the document $d$,
 * $n_d$ is defined as $n_d \equiv \sum_w n_{d,w}$,
 * $m_w$ is the number of documents containing the word $w$, and
 * $m$ is the total number of documents.

### TF-IDFの式のバリエーション

* https://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html


### 式の選び方
* どの式の形がいいかは、downstream taskの性能で選ぶ。
* どんな場合でもこれが一番良い、という式は、ない。

### scikit-learnのTfidfVectorizer
* scikit-learnでTF-IDFの計算式がどうなっているかは下記を参照。
  * https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

* デフォルトの設定を確認
  * https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

* 次のパラメータは、ちゃんと考えて設定した方が良い。
  * `max_df`, `min_df`, `stop_words`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
X = tfidf_vectorizer.fit_transform(corpus).toarray()
X

* テキストベクトルはL2ノルムが1となるように長さを変更されている。
  * TfidfVectorizer()のnormパラメータで変更可能。


In [ ]:
import numpy as np
np.linalg.norm(X, axis=1)

* 語彙の取得

In [ ]:
vocab = tfidf_vectorizer.get_feature_names_out()
vocab

* 新しいテキストをベクトル化
  * 新出単語は無視される（OoV問題）。

In [ ]:
new_vectors = tfidf_vectorizer.transform(new_doc).toarray()
new_vectors

* 各単語のIDF
  * IDFはそれぞれの単語について一意に決まる値。
  * テキストごとに求まる値ではない。
  * コーパスが変わると、IDFも変わる。

In [ ]:
tfidf_vectorizer.idf_

## BoWの応用

### テキスト間の類似度計算

* 内積による類似度計算


In [ ]:
np.dot(X, new_vectors[0])

* TfidfVectorizorのデフォルトの設定では、TF-IDFベクトルが長さ1にnormalizeされている。
* そのため、内積がコサイン類似度に一致する。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X, new_vectors)

* 問: テキストをベクトルとして表現する方法が分かった。これを使うと何ができるか？

## 演習: 20 newsgroups データセット
* テキスト分類手法の評価に使う、古典的なデータセット。

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()
y_true = newsgroups.target

* 下記コードを参考にして、数値を全て「#NUMBER」という特殊な単語へ変換する。
 * https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html#sphx-glr-auto-examples-bicluster-plot-bicluster-newsgroups-py

In [ ]:
def number_normalizer(tokens):
  return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

class NumberNormalizingVectorizer(TfidfVectorizer):
  def build_tokenizer(self):
    tokenizer = super().build_tokenizer()
    return lambda doc: list(number_normalizer(tokenizer(doc)))

In [ ]:
vectorizer = NumberNormalizingVectorizer(stop_words='english', min_df=5)

In [ ]:
X = vectorizer.fit_transform(newsgroups.data)

In [ ]:
print(vectorizer.get_feature_names_out()[:20])

In [ ]:
vocab = vectorizer.get_feature_names_out()
len(vocab)

* 密(dense)な配列に変換する。

In [ ]:
X = X.toarray()

* 適当な二つのテキストの内積を求めてみる。

In [ ]:
np.dot(X[0], X[1])

* 対応するクラスラベルを調べる。

In [ ]:
y_true[0], y_true[1]

In [ ]:
newsgroups.target_names

# 本日の課題: 簡単な情報検索
* TF-IDFベクトルを使って、20 newsgroupsのどれか一つのテキストについて・・・
* それと最も似ているテキストを10個返す関数を書こう。
* 10個のうち、元のテキストと同じクラスに属するテキストがいくつあるかを調べよう。
* 全てのテキストについて同じことを行なってみよう。そして・・・
* 最も似ている上位10個のうち同じクラスのテキスト数の平均値を求めよう。

## 補足: BM25

In [ ]:
!pip install bm25s

In [ ]:
import bm25s

retriever = bm25s.BM25(corpus=newsgroups.data)
retriever.index(bm25s.tokenize(newsgroups.data))

In [ ]:
query = newsgroups.data[0]
print("Query " + "="*60 + "\n" + query)

results, scores = retriever.retrieve(bm25s.tokenize(query), k=5)

for i in range(results.shape[1]):
  doc, score = results[0, i], scores[0, i]
  print(f"Rank {i+1} (score: {score:.2f})" + "-"*40)
  print(doc)